<a href="https://colab.research.google.com/github/Leon-Av/whisper_minicolab/blob/main/whisper_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка Whisper и необходимых зависимостей
!pip install -U openai-whisper
!apt-get update && apt-get install -y ffmpeg

import os
import whisper
import torch
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# Виджет для выбора скорости
speed_options = {
    'fast': 'small',
    'medium': 'medium',
    'slow': 'large'
}
speed_dropdown = widgets.Dropdown(
    options=[('Fast', 'fast'), ('Medium', 'medium'), ('Slow', 'slow')],
    value='fast',
    description='Speed:',
)

# Виджет для загрузки файла
upload_widget = widgets.FileUpload(
    accept='.mp3,.wav,.m4a,.mp4,.flac',  # поддерживаемые форматы
    multiple=False
)

# Кнопка запуска
run_button = widgets.Button(
    description="Transcribe",
    button_style='success'
)

# Вывод для отображения результатов
output = widgets.Output()

def transcribe_audio(b):
    with output:
        clear_output()
        if not upload_widget.value:
            print("Пожалуйста, загрузите аудиофайл.")
            return
        # Получаем имя и содержимое файла
        uploaded_filename = list(upload_widget.value.keys())[0]
        content = upload_widget.value[uploaded_filename]['content']
        # Сохраняем файл
        with open(uploaded_filename, 'wb') as f:
            f.write(content)
        # Определяем модель
        model_size = speed_options[speed_dropdown.value]
        # Проверяем доступность CUDA
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Выполняется команда: whisper {uploaded_filename} --model {model_size} --device {device}")
        # Загружаем и используем модель
        model = whisper.load_model(model_size, device=device)
        result = model.transcribe(uploaded_filename)
        print("Результат распознавания:")
        print(result["text"])

run_button.on_click(transcribe_audio)

# Отображаем виджеты
display(speed_dropdown)
display(upload_widget)
display(run_button)
display(output)
